In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
#from mlxtend.frequent_patterns import association_rules, apriori
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel, euclidean_distances
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer


#https://www.kaggle.com/rmisra/clothing-fit-dataset-for-size-recommendation?select=renttherunway_final_data.json

In [2]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise.model_selection import KFold
from surprise import Reader

In [3]:
df_train = pd.read_csv("X_train.csv")
df_test = pd.read_csv("X_test.csv")

In [4]:
df_train.head()

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date
0,fit,13906,34b,2616422,115lbs,10.0,everyday,its loose fitting in a good way,straight & narrow,"loved it, fun blouse, super pretty and easy to...",top,"5' 7""",8,32.0,"January 3, 2018"
1,fit,18490,32c,152836,115lbs,8.0,date,This dress was amazing! It fit like a glove a...,petite,Love it!,mini,"5' 0""",4,37.0,"March 30, 2014"
2,fit,476969,32b,176490,130lbs,10.0,wedding,I loved wearing this dress. It is really struc...,athletic,Love it.,dress,"5' 7""",8,34.0,"April 21, 2016"
3,fit,542220,34c,1777332,138lbs,8.0,work,It's a little boxy and very warm as it is wool...,athletic,"Classy, conservative work dress.",dress,"5' 7""",12,32.0,"February 10, 2016"
4,fit,334094,34a,1547971,130lbs,10.0,wedding,Rented this dress for a black-tie wedding and ...,straight & narrow,"Elegant, interesting, glamorous… can I keep it?",gown,"5' 6""",8,32.0,"March 25, 2016"


In [5]:
df_train["fit"].value_counts()

fit      57640
small    10851
large    10664
Name: fit, dtype: int64

# Data Cleansing

Para este data cleansing se siguio las recomendaciones de https://github.com/PrestonMWF/Rent-the-Runway-Recommender-System/blob/master/Rent_the_Runway_Customer_Analysis_Recommender_System.ipynb

In [6]:
#Esto no hace nada porque ya habiamos quitado los N/A en ingenieria de datos pero tenemos que ver si podemos dejarlo.
#Este codigo lo que hace es que para los rented for faltantes les mete el mas usado.
df_train["rented for"].fillna(df_train["rented for"].value_counts().index[0], inplace=True)
df_train.head()

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date
0,fit,13906,34b,2616422,115lbs,10.0,everyday,its loose fitting in a good way,straight & narrow,"loved it, fun blouse, super pretty and easy to...",top,"5' 7""",8,32.0,"January 3, 2018"
1,fit,18490,32c,152836,115lbs,8.0,date,This dress was amazing! It fit like a glove a...,petite,Love it!,mini,"5' 0""",4,37.0,"March 30, 2014"
2,fit,476969,32b,176490,130lbs,10.0,wedding,I loved wearing this dress. It is really struc...,athletic,Love it.,dress,"5' 7""",8,34.0,"April 21, 2016"
3,fit,542220,34c,1777332,138lbs,8.0,work,It's a little boxy and very warm as it is wool...,athletic,"Classy, conservative work dress.",dress,"5' 7""",12,32.0,"February 10, 2016"
4,fit,334094,34a,1547971,130lbs,10.0,wedding,Rented this dress for a black-tie wedding and ...,straight & narrow,"Elegant, interesting, glamorous… can I keep it?",gown,"5' 6""",8,32.0,"March 25, 2016"


In [7]:
#Cambia la fecha de review a fecha para que podamos manejarla. Esto no lo usamos pero la dejo por aca.
df_train["review_date"] = pd.to_datetime(df_train["review_date"])
df_train.head()

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date
0,fit,13906,34b,2616422,115lbs,10.0,everyday,its loose fitting in a good way,straight & narrow,"loved it, fun blouse, super pretty and easy to...",top,"5' 7""",8,32.0,2018-01-03
1,fit,18490,32c,152836,115lbs,8.0,date,This dress was amazing! It fit like a glove a...,petite,Love it!,mini,"5' 0""",4,37.0,2014-03-30
2,fit,476969,32b,176490,130lbs,10.0,wedding,I loved wearing this dress. It is really struc...,athletic,Love it.,dress,"5' 7""",8,34.0,2016-04-21
3,fit,542220,34c,1777332,138lbs,8.0,work,It's a little boxy and very warm as it is wool...,athletic,"Classy, conservative work dress.",dress,"5' 7""",12,32.0,2016-02-10
4,fit,334094,34a,1547971,130lbs,10.0,wedding,Rented this dress for a black-tie wedding and ...,straight & narrow,"Elegant, interesting, glamorous… can I keep it?",gown,"5' 6""",8,32.0,2016-03-25


In [8]:
#Quita los caracteres especiales de los review y los deja como espacios
df_train["review_text"] = df_train.review_text.str.replace("[^0-9a-zA-Z]+", " ")
df_train["review_summary"] = df_train.review_summary.str.replace("[^0-9a-zA-Z]+", " ")

In [9]:
df_train["review_summary"].count()

79026

In [10]:
#Me di cuenta que hay mucha palabra con numero en las review , cuando a nosotros nos interesa las palabras qeu identifican al item. Por estem motivo quito las palbaras que tienen numeros y pruebo.
df_train["review_text"] = df_train.review_text.str.replace("^\d+\s|\s\d+\s|\s\d+$", " ")
df_train["review_summary"] = df_train.review_summary.str.replace("^\d+\s|\s\d+\s|\s\d+$", " ")

In [11]:
df_train["review_text"]

0                          its loose fitting in a good way
1        This dress was amazing It fit like a glove and...
2        I loved wearing this dress It is really struct...
3        It s a little boxy and very warm as it is wool...
4        Rented this dress for a black tie wedding and ...
                               ...                        
79150    Very pretty Next time I d go with a size inste...
79151    This dress received the most compliments award...
79152    Wonderful dress to make an event statement As ...
79153    It was perfect for the cocktail NYE party I at...
79154    I wore this with a lace bralette because the t...
Name: review_text, Length: 79155, dtype: object

In [12]:
#Stemming
#https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-world-dataset-796d339a4089
from nltk.stem import PorterStemmer

porter_stemmer = PorterStemmer()

def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

df_train2 = df_train["review_text"][1:4].apply(stem_sentences)

df_train2

1    thi dress wa amaz It fit like a glove and wa t...
2    I love wear thi dress It is realli structur du...
3    It s a littl boxi and veri warm as it is wool ...
Name: review_text, dtype: object

## New content (Metadata created with category, review_summary and item_review_text)

In [13]:
#nltk.download('stopwords')


In [14]:
stop_words = stopwords.words("english")

df_train["item_review_text"] = (df_train["category"] + 
                                    #" " +
                                    #df_train["rented for"] +  " "  + df_train["body type"] + #No aporto nada.
                                    " " + 
                                    df_train["review_summary"] +
 
                                    " " +
                                    df_train["review_text"]).str.lower()
        


df_train["item_review_text"] = df_train["item_review_text"].astype(str)
df_train["item_review_text"] = df_train["item_review_text"].apply(
    lambda x: " ".join([word for word in x.split() if word not in (stop_words)])
)

df_train[["item_id", "item_review_text"]].tail()

,item_id,item_review_text
79150,2210425,top work got compliments pretty next time go s...
79151,762257,dress felt like celebrity dress received compl...
79152,221704,dress dazzling wonderful dress make event stat...
79153,1814016,dress great color perfect cocktail nye party a...
79154,1743343,dress rented twice work last fall date summer ...


In [15]:
agg_item_text = pd.DataFrame(df_train.groupby("item_id")["item_review_text"].sum())
agg_item_text.head()

,item_review_text
item_id,
123373,gown simple elegant warned loved dress fit wel...
123793,gown glam miles fashion inspo holiday party os...
124204,dress loads compliments night long stood among...
124553,dress love one flattering great color get enou...
125424,dress dress amazing hourglass figures flatteri...


In [16]:
agg_item_text = agg_item_text.reset_index()
agg_item_text

,item_id,item_review_text
0,123373,gown simple elegant warned loved dress fit wel...
1,123793,gown glam miles fashion inspo holiday party os...
2,124204,dress loads compliments night long stood among...
3,124553,dress love one flattering great color get enou...
4,125424,dress dress amazing hourglass figures flatteri...
...,...,...
5582,2963850,skirt loved fit comfortable loved crazy patter...
5583,2964470,top perfect sweater loved sweater tory burch f...
5584,2965009,coat omg fun fur coat fun wear actually warmer...
5585,2965924,jacket really unique yet top creative based re...


# Lemmatization

In [17]:
#Lo probe pero el mejor efecto me lo genero aplicar solo stemming
# nltk.download('wordnet')

# w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
# lemmatizer = nltk.stem.WordNetLemmatizer()

# def lemmatize_text(text):
#     return ' '.join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)])

# agg_item_text["item_review_text"] = agg_item_text["item_review_text"].apply(lemmatize_text)

# agg_item_text["item_review_text"]

# Stemming

In [18]:
#Stemming
#https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-world-dataset-796d339a4089
from nltk.stem import PorterStemmer

porter_stemmer = PorterStemmer()

def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

agg_item_text["item_review_text"] = agg_item_text["item_review_text"].apply(stem_sentences)

agg_item_text["item_review_text"]

0       gown simpl eleg warn love dress fit well beaut...
1       gown glam mile fashion inspo holiday parti osc...
2       dress load compliment night long stood amongst...
3       dress love one flatter great color get enough ...
4       dress dress amaz hourglass figur flatter tri t...
                              ...                        
5582    skirt love fit comfort love crazi pattern look...
5583    top perfect sweater love sweater tori burch fi...
5584    coat omg fun fur coat fun wear actual warmer e...
5585    jacket realli uniqu yet top creativ base revie...
5586    pant favorit pant ever love pant comfi cute wo...
Name: item_review_text, Length: 5587, dtype: object

### Item_review_text vectorization

In [19]:
#Prueba con ngram_range el mejor resultado es con (0,2) (ya probe con muchos)
#Este  strip_accents= "unicode" no sirvio para nada porque el acento lo toma bien.

tf_idf = TfidfVectorizer(ngram_range = (0,2))
tf_idf_matrix = tf_idf.fit_transform(agg_item_text["item_review_text"])

# SVD training

In [20]:
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 10))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df_train[['user_id', 'item_id', 'rating']], reader)

# We can now use this dataset as we please, e.g. calling cross_validate
cross_validate(NormalPredictor(), data, cv=2)

# define a cross-validation iterator
kf = KFold(n_splits=10, random_state=14)

svd = SVD(n_factors=10, lr_all=0.0104, n_epochs=5, reg_all=0.1378) 

for trainset, testset in kf.split(data):

    # train and test algorithm.
    svd.fit(trainset)
    predictions = svd.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

RMSE: 1.4602
RMSE: 1.4124
RMSE: 1.4389
RMSE: 1.4427
RMSE: 1.4477
RMSE: 1.4302
RMSE: 1.4402
RMSE: 1.4520
RMSE: 1.4246
RMSE: 1.4301


## Recommendation function

In [21]:
#Construct a reverse mapping of indices and movie titles
indices = pd.Series(agg_item_text.index, index=agg_item_text['item_id'])


In [22]:
#Me quedo con los usuarios que dieron raiting alto para comparar contra mis predicciones
X_test = df_test[df_test["rating"] >= 8]

In [23]:
#Nos quedamos con los usuarios unicos de test para predecir raitings con items
users_test_unique = X_test["user_id"].unique()
len(users_test_unique)

23721

In [24]:
df_train_best_ratings = df_train.sort_values(by= "rating",ascending= False)

In [25]:
def content_recommender_generic(item_id, distance_matrix, n, df=agg_item_text, indices=indices):
    # Obtain the index of the movie that matches the title
    idx = indices[item_id]
    
    # Get the pairwsie similarity scores of all movies with that movie
    # And convert it into a list of tuples as described above
    
    sim_scores = list(enumerate(distance_matrix[idx]))
    
    # Sort the movies based on the cosine similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the scores of the (n-1) most similar movies. Ignore the first movie.
    sim_scores = sim_scores[1:n]
    
    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    
    # Return the top 10 most similar movies
    return agg_item_text['item_id'].iloc[movie_indices]

In [26]:
#Usuarios que le voy a aplicar el collaborative. Para este caso son todos los usuarios
users_to_apply_collaborative = df_train["user_id"].value_counts()[df_train["user_id"].value_counts() >= 0]
len(users_to_apply_collaborative)

31321

In [27]:
def n_hybrid_recommendation(users_test_unique, n , distance_matrix):
    n = n
    reco=[]
    distance_matrix = distance_matrix
    count_user = 0
    count_user_colab = 0
    for user_id in users_test_unique: 
        #Me quedo con el primer item relevante del usuario en train
        item_id_content = df_train_best_ratings[df_train_best_ratings["user_id"] == user_id]["item_id"].values[0]
        #Aplico la funcion de 
        content_recommendation = pd.DataFrame(content_recommender_generic(item_id=item_id_content, distance_matrix=distance_matrix, n=n))
        content_recommendation["user_id"] = user_id
        
        if user_id in users_to_apply_collaborative:
            predict = []
            for item_id in content_recommendation["item_id"]:
                predict.append(svd.predict(user_id, item_id)[3])
            content_recommendation['pred_rating'] = predict
            content_recommendation = content_recommendation.sort_values(by=["pred_rating"],ascending=False)
            reco.append(content_recommendation["item_id"].to_list())
            count_user_colab += 1
        
        else:
            reco.append(content_recommendation["item_id"].to_list())
        
        count_user += 1     
        print("Progress in % {:.2f} " .format((count_user /len(users_test_unique) * 100)))
    
    return reco, count_user_colab

In [28]:
len(users_test_unique)

23721

### Score metrics (precision, recall and F1)

In [29]:
reco_all_user=[]

In [30]:
def score_metrics( n=6, reco_all_user=reco_all_user):
    TP = 0
    FN = 0
    for i in range(len(reco_all_user)):
        user = users_test_unique[i]
        list_test_item_per_user = X_test[X_test["user_id"]==user]["item_id"].values
        reco_user = reco_all_user[i]
        for item in list_test_item_per_user:      
            if item in reco_user:
                TP += 1
            else:
                FN += 1
    precision = round((TP)/((n-1)*len(users_test_unique)),5) #menos 1 porque el n debe ser n+1
    recall = round((TP)/(TP+FN),5)
    f1 = 2*((precision * recall)/(precision + recall))
    return precision, recall, f1

### Cosine distance 

In [31]:
#Columnas del dataframe para entrenar
print("Cantidad de palabras a utilizar: ",len(tf_idf.get_feature_names()))
print(tf_idf.get_feature_names())

Cantidad de palabras a utilizar:  582430


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [32]:
tf_idf.get_feature_names()[60:20000]

['0gown',
 '0gown eleg',
 '0jumpsuit',
 '0jumpsuit great',
 '0l',
 '0l 0r',
 '0l 10',
 '0l 2l',
 '0l 2r',
 '0l accord',
 '0l alreadi',
 '0l avail',
 '0l availablegown',
 '0l back',
 '0l backup',
 '0l becuas',
 '0l better',
 '0l big',
 '0l bit',
 '0l came',
 '0l could',
 '0l current',
 '0l definit',
 '0l dress',
 '0l end',
 '0l even',
 '0l fit',
 '0l flatter',
 '0l found',
 '0l futuregown',
 '0l gave',
 '0l great',
 '0l hit',
 '0l idea',
 '0l inch',
 '0l instead',
 '0l key',
 '0l lace',
 '0l lb',
 '0l like',
 '0l long',
 '0l longer',
 '0l lovedgown',
 '0l low',
 '0l materi',
 '0l much',
 '0l peplum',
 '0l perfect',
 '0l probabl',
 '0l regrettedgown',
 '0l rip',
 '0l seem',
 '0l sent',
 '0l short',
 '0l sinc',
 '0l six',
 '0l size',
 '0l slight',
 '0l small',
 '0l smidg',
 '0l snug',
 '0l someon',
 '0l still',
 '0l stock',
 '0l tad',
 '0l take',
 '0l think',
 '0l tight',
 '0l true',
 '0l typic',
 '0l ultim',
 '0l unavail',
 '0l usual',
 '0l way',
 '0l wear',
 '0l weigh',
 '0l went',
 '0l

In [33]:
#Data frame que usa para entrenar
#pd.DataFrame(tf_idf_matrix.toarray())

In [34]:
item_tfidf_similarity = cosine_similarity(tf_idf_matrix, tf_idf_matrix)
item_tfidf_similarity_df = pd.DataFrame(item_tfidf_similarity,
                                        columns=agg_item_text.index,
                                        index=agg_item_text.index)

item_tfidf_similarity_df.head()


,0,1,2,3,4,5,6,7,8,9,...,5577,5578,5579,5580,5581,5582,5583,5584,5585,5586
0,1.000000,0.991228,0.986409,0.987650,0.987411,0.989892,0.988332,0.989884,0.984185,0.986524,...,0.882010,0.536200,0.853660,0.796914,0.469210,0.789405,0.570304,0.708503,0.843467,0.809412
1,0.991228,1.000000,0.989998,0.989127,0.988778,0.992421,0.990115,0.993818,0.986397,0.987169,...,0.882490,0.535742,0.854965,0.797745,0.468573,0.790248,0.570387,0.709901,0.843788,0.809704
2,0.986409,0.989998,1.000000,0.987473,0.987087,0.987333,0.985014,0.992935,0.981320,0.986866,...,0.880328,0.535176,0.852773,0.797776,0.468049,0.788719,0.570425,0.707862,0.841561,0.806619
3,0.987650,0.989127,0.987473,1.000000,0.989612,0.989442,0.985760,0.990652,0.982676,0.987141,...,0.882949,0.536708,0.855120,0.798897,0.470952,0.790365,0.571226,0.708350,0.843644,0.807279
4,0.987411,0.988778,0.987087,0.989612,1.000000,0.987548,0.985567,0.990339,0.982132,0.986589,...,0.883303,0.536989,0.854866,0.798198,0.470022,0.790497,0.571335,0.708141,0.844302,0.806846


In [35]:
len(item_tfidf_similarity)

5587

### 5 recommendations

In [36]:
reco_all_user_5_cos = []
reco_all_user_5_cos, count_user_colab = n_hybrid_recommendation(users_test_unique, n=6, distance_matrix=item_tfidf_similarity)

Progress in % 0.00 
Progress in % 0.01 
Progress in % 0.01 
Progress in % 0.02 
Progress in % 0.02 
Progress in % 0.03 
Progress in % 0.03 
Progress in % 0.03 
Progress in % 0.04 
Progress in % 0.04 
Progress in % 0.05 
Progress in % 0.05 
Progress in % 0.05 
Progress in % 0.06 
Progress in % 0.06 
Progress in % 0.07 
Progress in % 0.07 
Progress in % 0.08 
Progress in % 0.08 
Progress in % 0.08 
Progress in % 0.09 
Progress in % 0.09 
Progress in % 0.10 
Progress in % 0.10 
Progress in % 0.11 
Progress in % 0.11 
Progress in % 0.11 
Progress in % 0.12 
Progress in % 0.12 
Progress in % 0.13 
Progress in % 0.13 
Progress in % 0.13 
Progress in % 0.14 
Progress in % 0.14 
Progress in % 0.15 
Progress in % 0.15 
Progress in % 0.16 
Progress in % 0.16 
Progress in % 0.16 
Progress in % 0.17 
Progress in % 0.17 
Progress in % 0.18 
Progress in % 0.18 
Progress in % 0.19 
Progress in % 0.19 
Progress in % 0.19 
Progress in % 0.20 
Progress in % 0.20 
Progress in % 0.21 
Progress in % 0.21 


Progress in % 1.76 
Progress in % 1.77 
Progress in % 1.77 
Progress in % 1.77 
Progress in % 1.78 
Progress in % 1.78 
Progress in % 1.79 
Progress in % 1.79 
Progress in % 1.80 
Progress in % 1.80 
Progress in % 1.80 
Progress in % 1.81 
Progress in % 1.81 
Progress in % 1.82 
Progress in % 1.82 
Progress in % 1.83 
Progress in % 1.83 
Progress in % 1.83 
Progress in % 1.84 
Progress in % 1.84 
Progress in % 1.85 
Progress in % 1.85 
Progress in % 1.85 
Progress in % 1.86 
Progress in % 1.86 
Progress in % 1.87 
Progress in % 1.87 
Progress in % 1.88 
Progress in % 1.88 
Progress in % 1.88 
Progress in % 1.89 
Progress in % 1.89 
Progress in % 1.90 
Progress in % 1.90 
Progress in % 1.91 
Progress in % 1.91 
Progress in % 1.91 
Progress in % 1.92 
Progress in % 1.92 
Progress in % 1.93 
Progress in % 1.93 
Progress in % 1.93 
Progress in % 1.94 
Progress in % 1.94 
Progress in % 1.95 
Progress in % 1.95 
Progress in % 1.96 
Progress in % 1.96 
Progress in % 1.96 
Progress in % 1.97 


Progress in % 3.54 
Progress in % 3.55 
Progress in % 3.55 
Progress in % 3.55 
Progress in % 3.56 
Progress in % 3.56 
Progress in % 3.57 
Progress in % 3.57 
Progress in % 3.57 
Progress in % 3.58 
Progress in % 3.58 
Progress in % 3.59 
Progress in % 3.59 
Progress in % 3.60 
Progress in % 3.60 
Progress in % 3.60 
Progress in % 3.61 
Progress in % 3.61 
Progress in % 3.62 
Progress in % 3.62 
Progress in % 3.63 
Progress in % 3.63 
Progress in % 3.63 
Progress in % 3.64 
Progress in % 3.64 
Progress in % 3.65 
Progress in % 3.65 
Progress in % 3.65 
Progress in % 3.66 
Progress in % 3.66 
Progress in % 3.67 
Progress in % 3.67 
Progress in % 3.68 
Progress in % 3.68 
Progress in % 3.68 
Progress in % 3.69 
Progress in % 3.69 
Progress in % 3.70 
Progress in % 3.70 
Progress in % 3.71 
Progress in % 3.71 
Progress in % 3.71 
Progress in % 3.72 
Progress in % 3.72 
Progress in % 3.73 
Progress in % 3.73 
Progress in % 3.74 
Progress in % 3.74 
Progress in % 3.74 
Progress in % 3.75 


Progress in % 5.31 
Progress in % 5.32 
Progress in % 5.32 
Progress in % 5.32 
Progress in % 5.33 
Progress in % 5.33 
Progress in % 5.34 
Progress in % 5.34 
Progress in % 5.35 
Progress in % 5.35 
Progress in % 5.35 
Progress in % 5.36 
Progress in % 5.36 
Progress in % 5.37 
Progress in % 5.37 
Progress in % 5.37 
Progress in % 5.38 
Progress in % 5.38 
Progress in % 5.39 
Progress in % 5.39 
Progress in % 5.40 
Progress in % 5.40 
Progress in % 5.40 
Progress in % 5.41 
Progress in % 5.41 
Progress in % 5.42 
Progress in % 5.42 
Progress in % 5.43 
Progress in % 5.43 
Progress in % 5.43 
Progress in % 5.44 
Progress in % 5.44 
Progress in % 5.45 
Progress in % 5.45 
Progress in % 5.46 
Progress in % 5.46 
Progress in % 5.46 
Progress in % 5.47 
Progress in % 5.47 
Progress in % 5.48 
Progress in % 5.48 
Progress in % 5.48 
Progress in % 5.49 
Progress in % 5.49 
Progress in % 5.50 
Progress in % 5.50 
Progress in % 5.51 
Progress in % 5.51 
Progress in % 5.51 
Progress in % 5.52 


Progress in % 7.06 
Progress in % 7.06 
Progress in % 7.07 
Progress in % 7.07 
Progress in % 7.07 
Progress in % 7.08 
Progress in % 7.08 
Progress in % 7.09 
Progress in % 7.09 
Progress in % 7.09 
Progress in % 7.10 
Progress in % 7.10 
Progress in % 7.11 
Progress in % 7.11 
Progress in % 7.12 
Progress in % 7.12 
Progress in % 7.12 
Progress in % 7.13 
Progress in % 7.13 
Progress in % 7.14 
Progress in % 7.14 
Progress in % 7.15 
Progress in % 7.15 
Progress in % 7.15 
Progress in % 7.16 
Progress in % 7.16 
Progress in % 7.17 
Progress in % 7.17 
Progress in % 7.18 
Progress in % 7.18 
Progress in % 7.18 
Progress in % 7.19 
Progress in % 7.19 
Progress in % 7.20 
Progress in % 7.20 
Progress in % 7.20 
Progress in % 7.21 
Progress in % 7.21 
Progress in % 7.22 
Progress in % 7.22 
Progress in % 7.23 
Progress in % 7.23 
Progress in % 7.23 
Progress in % 7.24 
Progress in % 7.24 
Progress in % 7.25 
Progress in % 7.25 
Progress in % 7.26 
Progress in % 7.26 
Progress in % 7.26 


...

Progress in % 91.53 
Progress in % 91.53 
Progress in % 91.53 
Progress in % 91.54 
Progress in % 91.54 
Progress in % 91.55 
Progress in % 91.55 
Progress in % 91.56 
Progress in % 91.56 
Progress in % 91.56 
Progress in % 91.57 
Progress in % 91.57 
Progress in % 91.58 
Progress in % 91.58 
Progress in % 91.59 
Progress in % 91.59 
Progress in % 91.59 
Progress in % 91.60 
Progress in % 91.60 
Progress in % 91.61 
Progress in % 91.61 
Progress in % 91.62 
Progress in % 91.62 
Progress in % 91.62 
Progress in % 91.63 
Progress in % 91.63 
Progress in % 91.64 
Progress in % 91.64 
Progress in % 91.64 
Progress in % 91.65 
Progress in % 91.65 
Progress in % 91.66 
Progress in % 91.66 
Progress in % 91.67 
Progress in % 91.67 
Progress in % 91.67 
Progress in % 91.68 
Progress in % 91.68 
Progress in % 91.69 
Progress in % 91.69 
Progress in % 91.70 
Progress in % 91.70 
Progress in % 91.70 
Progress in % 91.71 
Progress in % 91.71 
Progress in % 91.72 
Progress in % 91.72 
Progress in %

Progress in % 94.89 
Progress in % 94.90 
Progress in % 94.90 
Progress in % 94.91 
Progress in % 94.91 
Progress in % 94.92 
Progress in % 94.92 
Progress in % 94.92 
Progress in % 94.93 
Progress in % 94.93 
Progress in % 94.94 
Progress in % 94.94 
Progress in % 94.95 
Progress in % 94.95 
Progress in % 94.95 
Progress in % 94.96 
Progress in % 94.96 
Progress in % 94.97 
Progress in % 94.97 
Progress in % 94.97 
Progress in % 94.98 
Progress in % 94.98 
Progress in % 94.99 
Progress in % 94.99 
Progress in % 95.00 
Progress in % 95.00 
Progress in % 95.00 
Progress in % 95.01 
Progress in % 95.01 
Progress in % 95.02 
Progress in % 95.02 
Progress in % 95.03 
Progress in % 95.03 
Progress in % 95.03 
Progress in % 95.04 
Progress in % 95.04 
Progress in % 95.05 
Progress in % 95.05 
Progress in % 95.06 
Progress in % 95.06 
Progress in % 95.06 
Progress in % 95.07 
Progress in % 95.07 
Progress in % 95.08 
Progress in % 95.08 
Progress in % 95.08 
Progress in % 95.09 
Progress in %

Progress in % 96.59 
Progress in % 96.59 
Progress in % 96.59 
Progress in % 96.60 
Progress in % 96.60 
Progress in % 96.61 
Progress in % 96.61 
Progress in % 96.61 
Progress in % 96.62 
Progress in % 96.62 
Progress in % 96.63 
Progress in % 96.63 
Progress in % 96.64 
Progress in % 96.64 
Progress in % 96.64 
Progress in % 96.65 
Progress in % 96.65 
Progress in % 96.66 
Progress in % 96.66 
Progress in % 96.67 
Progress in % 96.67 
Progress in % 96.67 
Progress in % 96.68 
Progress in % 96.68 
Progress in % 96.69 
Progress in % 96.69 
Progress in % 96.69 
Progress in % 96.70 
Progress in % 96.70 
Progress in % 96.71 
Progress in % 96.71 
Progress in % 96.72 
Progress in % 96.72 
Progress in % 96.72 
Progress in % 96.73 
Progress in % 96.73 
Progress in % 96.74 
Progress in % 96.74 
Progress in % 96.75 
Progress in % 96.75 
Progress in % 96.75 
Progress in % 96.76 
Progress in % 96.76 
Progress in % 96.77 
Progress in % 96.77 
Progress in % 96.78 
Progress in % 96.78 
Progress in %

Progress in % 98.28 
Progress in % 98.29 
Progress in % 98.29 
Progress in % 98.30 
Progress in % 98.30 
Progress in % 98.31 
Progress in % 98.31 
Progress in % 98.31 
Progress in % 98.32 
Progress in % 98.32 
Progress in % 98.33 
Progress in % 98.33 
Progress in % 98.33 
Progress in % 98.34 
Progress in % 98.34 
Progress in % 98.35 
Progress in % 98.35 
Progress in % 98.36 
Progress in % 98.36 
Progress in % 98.36 
Progress in % 98.37 
Progress in % 98.37 
Progress in % 98.38 
Progress in % 98.38 
Progress in % 98.39 
Progress in % 98.39 
Progress in % 98.39 
Progress in % 98.40 
Progress in % 98.40 
Progress in % 98.41 
Progress in % 98.41 
Progress in % 98.41 
Progress in % 98.42 
Progress in % 98.42 
Progress in % 98.43 
Progress in % 98.43 
Progress in % 98.44 
Progress in % 98.44 
Progress in % 98.44 
Progress in % 98.45 
Progress in % 98.45 
Progress in % 98.46 
Progress in % 98.46 
Progress in % 98.47 
Progress in % 98.47 
Progress in % 98.47 
Progress in % 98.48 
Progress in %

Progress in % 99.97 
Progress in % 99.97 
Progress in % 99.98 
Progress in % 99.98 
Progress in % 99.99 
Progress in % 99.99 
Progress in % 100.00 
Progress in % 100.00 


In [37]:
reco_all_user_5_cos[1:5]

[[123793, 127865, 132738, 145906, 126335],
 [730008, 136860, 136110, 126335, 174086],
 [127865, 132738, 126335, 136110, 174086],
 [136110, 126335, 174086, 172027, 124553]]

In [38]:
count_user_colab

23721

In [39]:
precision_5_cos, recall_5_cos, f1_5_cos = score_metrics(n=6, reco_all_user= reco_all_user_5_cos)

In [40]:
print(precision_5_cos)
print(recall_5_cos)
print(f1_5_cos)

0.00916
0.03503
0.014522507354605113


### 10 recommendations

In [41]:
reco_all_user_10_cos = []
reco_all_user_10_cos,_ = n_hybrid_recommendation(users_test_unique, n=11, distance_matrix=item_tfidf_similarity)

Progress in % 0.00 
Progress in % 0.01 
Progress in % 0.01 
Progress in % 0.02 
Progress in % 0.02 
Progress in % 0.03 
Progress in % 0.03 
Progress in % 0.03 
Progress in % 0.04 
Progress in % 0.04 
Progress in % 0.05 
Progress in % 0.05 
Progress in % 0.05 
Progress in % 0.06 
Progress in % 0.06 
Progress in % 0.07 
Progress in % 0.07 
Progress in % 0.08 
Progress in % 0.08 
Progress in % 0.08 
Progress in % 0.09 
Progress in % 0.09 
Progress in % 0.10 
Progress in % 0.10 
Progress in % 0.11 
Progress in % 0.11 
Progress in % 0.11 
Progress in % 0.12 
Progress in % 0.12 
Progress in % 0.13 
Progress in % 0.13 
Progress in % 0.13 
Progress in % 0.14 
Progress in % 0.14 
Progress in % 0.15 
Progress in % 0.15 
Progress in % 0.16 
Progress in % 0.16 
Progress in % 0.16 
Progress in % 0.17 
Progress in % 0.17 
Progress in % 0.18 
Progress in % 0.18 
Progress in % 0.19 
Progress in % 0.19 
Progress in % 0.19 
Progress in % 0.20 
Progress in % 0.20 
Progress in % 0.21 
Progress in % 0.21 


Progress in % 1.79 
Progress in % 1.80 
Progress in % 1.80 
Progress in % 1.80 
Progress in % 1.81 
Progress in % 1.81 
Progress in % 1.82 
Progress in % 1.82 
Progress in % 1.83 
Progress in % 1.83 
Progress in % 1.83 
Progress in % 1.84 
Progress in % 1.84 
Progress in % 1.85 
Progress in % 1.85 
Progress in % 1.85 
Progress in % 1.86 
Progress in % 1.86 
Progress in % 1.87 
Progress in % 1.87 
Progress in % 1.88 
Progress in % 1.88 
Progress in % 1.88 
Progress in % 1.89 
Progress in % 1.89 
Progress in % 1.90 
Progress in % 1.90 
Progress in % 1.91 
Progress in % 1.91 
Progress in % 1.91 
Progress in % 1.92 
Progress in % 1.92 
Progress in % 1.93 
Progress in % 1.93 
Progress in % 1.93 
Progress in % 1.94 
Progress in % 1.94 
Progress in % 1.95 
Progress in % 1.95 
Progress in % 1.96 
Progress in % 1.96 
Progress in % 1.96 
Progress in % 1.97 
Progress in % 1.97 
Progress in % 1.98 
Progress in % 1.98 
Progress in % 1.99 
Progress in % 1.99 
Progress in % 1.99 
Progress in % 2.00 


Progress in % 3.54 
Progress in % 3.55 
Progress in % 3.55 
Progress in % 3.55 
Progress in % 3.56 
Progress in % 3.56 
Progress in % 3.57 
Progress in % 3.57 
Progress in % 3.57 
Progress in % 3.58 
Progress in % 3.58 
Progress in % 3.59 
Progress in % 3.59 
Progress in % 3.60 
Progress in % 3.60 
Progress in % 3.60 
Progress in % 3.61 
Progress in % 3.61 
Progress in % 3.62 
Progress in % 3.62 
Progress in % 3.63 
Progress in % 3.63 
Progress in % 3.63 
Progress in % 3.64 
Progress in % 3.64 
Progress in % 3.65 
Progress in % 3.65 
Progress in % 3.65 
Progress in % 3.66 
Progress in % 3.66 
Progress in % 3.67 
Progress in % 3.67 
Progress in % 3.68 
Progress in % 3.68 
Progress in % 3.68 
Progress in % 3.69 
Progress in % 3.69 
Progress in % 3.70 
Progress in % 3.70 
Progress in % 3.71 
Progress in % 3.71 
Progress in % 3.71 
Progress in % 3.72 
Progress in % 3.72 
Progress in % 3.73 
Progress in % 3.73 
Progress in % 3.74 
Progress in % 3.74 
Progress in % 3.74 
Progress in % 3.75 


Progress in % 5.30 
Progress in % 5.30 
Progress in % 5.31 
Progress in % 5.31 
Progress in % 5.32 
Progress in % 5.32 
Progress in % 5.32 
Progress in % 5.33 
Progress in % 5.33 
Progress in % 5.34 
Progress in % 5.34 
Progress in % 5.35 
Progress in % 5.35 
Progress in % 5.35 
Progress in % 5.36 
Progress in % 5.36 
Progress in % 5.37 
Progress in % 5.37 
Progress in % 5.37 
Progress in % 5.38 
Progress in % 5.38 
Progress in % 5.39 
Progress in % 5.39 
Progress in % 5.40 
Progress in % 5.40 
Progress in % 5.40 
Progress in % 5.41 
Progress in % 5.41 
Progress in % 5.42 
Progress in % 5.42 
Progress in % 5.43 
Progress in % 5.43 
Progress in % 5.43 
Progress in % 5.44 
Progress in % 5.44 
Progress in % 5.45 
Progress in % 5.45 
Progress in % 5.46 
Progress in % 5.46 
Progress in % 5.46 
Progress in % 5.47 
Progress in % 5.47 
Progress in % 5.48 
Progress in % 5.48 
Progress in % 5.48 
Progress in % 5.49 
Progress in % 5.49 
Progress in % 5.50 
Progress in % 5.50 
Progress in % 5.51 


Progress in % 7.03 
Progress in % 7.04 
Progress in % 7.04 
Progress in % 7.04 
Progress in % 7.05 
Progress in % 7.05 
Progress in % 7.06 
Progress in % 7.06 
Progress in % 7.07 
Progress in % 7.07 
Progress in % 7.07 
Progress in % 7.08 
Progress in % 7.08 
Progress in % 7.09 
Progress in % 7.09 
Progress in % 7.09 
Progress in % 7.10 
Progress in % 7.10 
Progress in % 7.11 
Progress in % 7.11 
Progress in % 7.12 
Progress in % 7.12 
Progress in % 7.12 
Progress in % 7.13 
Progress in % 7.13 
Progress in % 7.14 
Progress in % 7.14 
Progress in % 7.15 
Progress in % 7.15 
Progress in % 7.15 
Progress in % 7.16 
Progress in % 7.16 
Progress in % 7.17 
Progress in % 7.17 
Progress in % 7.18 
Progress in % 7.18 
Progress in % 7.18 
Progress in % 7.19 
Progress in % 7.19 
Progress in % 7.20 
Progress in % 7.20 
Progress in % 7.20 
Progress in % 7.21 
Progress in % 7.21 
Progress in % 7.22 
Progress in % 7.22 
Progress in % 7.23 
Progress in % 7.23 
Progress in % 7.23 
Progress in % 7.24 


...

Progress in % 91.19 
Progress in % 91.19 
Progress in % 91.19 
Progress in % 91.20 
Progress in % 91.20 
Progress in % 91.21 
Progress in % 91.21 
Progress in % 91.21 
Progress in % 91.22 
Progress in % 91.22 
Progress in % 91.23 
Progress in % 91.23 
Progress in % 91.24 
Progress in % 91.24 
Progress in % 91.24 
Progress in % 91.25 
Progress in % 91.25 
Progress in % 91.26 
Progress in % 91.26 
Progress in % 91.27 
Progress in % 91.27 
Progress in % 91.27 
Progress in % 91.28 
Progress in % 91.28 
Progress in % 91.29 
Progress in % 91.29 
Progress in % 91.29 
Progress in % 91.30 
Progress in % 91.30 
Progress in % 91.31 
Progress in % 91.31 
Progress in % 91.32 
Progress in % 91.32 
Progress in % 91.32 
Progress in % 91.33 
Progress in % 91.33 
Progress in % 91.34 
Progress in % 91.34 
Progress in % 91.35 
Progress in % 91.35 
Progress in % 91.35 
Progress in % 91.36 
Progress in % 91.36 
Progress in % 91.37 
Progress in % 91.37 
Progress in % 91.37 
Progress in % 91.38 
Progress in %

Progress in % 92.88 
Progress in % 92.89 
Progress in % 92.89 
Progress in % 92.90 
Progress in % 92.90 
Progress in % 92.91 
Progress in % 92.91 
Progress in % 92.91 
Progress in % 92.92 
Progress in % 92.92 
Progress in % 92.93 
Progress in % 92.93 
Progress in % 92.93 
Progress in % 92.94 
Progress in % 92.94 
Progress in % 92.95 
Progress in % 92.95 
Progress in % 92.96 
Progress in % 92.96 
Progress in % 92.96 
Progress in % 92.97 
Progress in % 92.97 
Progress in % 92.98 
Progress in % 92.98 
Progress in % 92.99 
Progress in % 92.99 
Progress in % 92.99 
Progress in % 93.00 
Progress in % 93.00 
Progress in % 93.01 
Progress in % 93.01 
Progress in % 93.01 
Progress in % 93.02 
Progress in % 93.02 
Progress in % 93.03 
Progress in % 93.03 
Progress in % 93.04 
Progress in % 93.04 
Progress in % 93.04 
Progress in % 93.05 
Progress in % 93.05 
Progress in % 93.06 
Progress in % 93.06 
Progress in % 93.07 
Progress in % 93.07 
Progress in % 93.07 
Progress in % 93.08 
Progress in %

Progress in % 94.57 
Progress in % 94.57 
Progress in % 94.57 
Progress in % 94.58 
Progress in % 94.58 
Progress in % 94.59 
Progress in % 94.59 
Progress in % 94.60 
Progress in % 94.60 
Progress in % 94.60 
Progress in % 94.61 
Progress in % 94.61 
Progress in % 94.62 
Progress in % 94.62 
Progress in % 94.63 
Progress in % 94.63 
Progress in % 94.63 
Progress in % 94.64 
Progress in % 94.64 
Progress in % 94.65 
Progress in % 94.65 
Progress in % 94.65 
Progress in % 94.66 
Progress in % 94.66 
Progress in % 94.67 
Progress in % 94.67 
Progress in % 94.68 
Progress in % 94.68 
Progress in % 94.68 
Progress in % 94.69 
Progress in % 94.69 
Progress in % 94.70 
Progress in % 94.70 
Progress in % 94.71 
Progress in % 94.71 
Progress in % 94.71 
Progress in % 94.72 
Progress in % 94.72 
Progress in % 94.73 
Progress in % 94.73 
Progress in % 94.73 
Progress in % 94.74 
Progress in % 94.74 
Progress in % 94.75 
Progress in % 94.75 
Progress in % 94.76 
Progress in % 94.76 
Progress in %

Progress in % 96.29 
Progress in % 96.29 
Progress in % 96.30 
Progress in % 96.30 
Progress in % 96.31 
Progress in % 96.31 
Progress in % 96.32 
Progress in % 96.32 
Progress in % 96.32 
Progress in % 96.33 
Progress in % 96.33 
Progress in % 96.34 
Progress in % 96.34 
Progress in % 96.35 
Progress in % 96.35 
Progress in % 96.35 
Progress in % 96.36 
Progress in % 96.36 
Progress in % 96.37 
Progress in % 96.37 
Progress in % 96.37 
Progress in % 96.38 
Progress in % 96.38 
Progress in % 96.39 
Progress in % 96.39 
Progress in % 96.40 
Progress in % 96.40 
Progress in % 96.40 
Progress in % 96.41 
Progress in % 96.41 
Progress in % 96.42 
Progress in % 96.42 
Progress in % 96.43 
Progress in % 96.43 
Progress in % 96.43 
Progress in % 96.44 
Progress in % 96.44 
Progress in % 96.45 
Progress in % 96.45 
Progress in % 96.45 
Progress in % 96.46 
Progress in % 96.46 
Progress in % 96.47 
Progress in % 96.47 
Progress in % 96.48 
Progress in % 96.48 
Progress in % 96.48 
Progress in %

Progress in % 99.61 
Progress in % 99.62 
Progress in % 99.62 
Progress in % 99.62 
Progress in % 99.63 
Progress in % 99.63 
Progress in % 99.64 
Progress in % 99.64 
Progress in % 99.65 
Progress in % 99.65 
Progress in % 99.65 
Progress in % 99.66 
Progress in % 99.66 
Progress in % 99.67 
Progress in % 99.67 
Progress in % 99.68 
Progress in % 99.68 
Progress in % 99.68 
Progress in % 99.69 
Progress in % 99.69 
Progress in % 99.70 
Progress in % 99.70 
Progress in % 99.70 
Progress in % 99.71 
Progress in % 99.71 
Progress in % 99.72 
Progress in % 99.72 
Progress in % 99.73 
Progress in % 99.73 
Progress in % 99.73 
Progress in % 99.74 
Progress in % 99.74 
Progress in % 99.75 
Progress in % 99.75 
Progress in % 99.76 
Progress in % 99.76 
Progress in % 99.76 
Progress in % 99.77 
Progress in % 99.77 
Progress in % 99.78 
Progress in % 99.78 
Progress in % 99.79 
Progress in % 99.79 
Progress in % 99.79 
Progress in % 99.80 
Progress in % 99.80 
Progress in % 99.81 
Progress in %

In [42]:
reco_all_user_10_cos[1:5]

[[123793,
  127865,
  131533,
  125465,
  131117,
  132738,
  145906,
  126335,
  154002,
  130259],
 [127865,
  730008,
  136860,
  132738,
  147594,
  136110,
  126335,
  144051,
  174086,
  172027],
 [123793,
  137585,
  127865,
  136860,
  132738,
  145906,
  126335,
  136110,
  174086,
  172027],
 [127865,
  131533,
  136110,
  126335,
  174086,
  148089,
  152836,
  172027,
  124553,
  166633]]

In [43]:
precision_10_cos, recall_10_cos, f1_10_cos = score_metrics(n=11, reco_all_user= reco_all_user_10_cos)

In [44]:
print(precision_10_cos)
print(recall_10_cos)
print(f1_10_cos)

0.0072
0.05506
0.012734725345326051


## 20 recommendations

In [45]:
reco_all_user_20_cos = []
reco_all_user_20_cos, _ = n_hybrid_recommendation(users_test_unique, n=21, distance_matrix=item_tfidf_similarity)

Progress in % 0.00 
Progress in % 0.01 
Progress in % 0.01 
Progress in % 0.02 
Progress in % 0.02 
Progress in % 0.03 
Progress in % 0.03 
Progress in % 0.03 
Progress in % 0.04 
Progress in % 0.04 
Progress in % 0.05 
Progress in % 0.05 
Progress in % 0.05 
Progress in % 0.06 
Progress in % 0.06 
Progress in % 0.07 
Progress in % 0.07 
Progress in % 0.08 
Progress in % 0.08 
Progress in % 0.08 
Progress in % 0.09 
Progress in % 0.09 
Progress in % 0.10 
Progress in % 0.10 
Progress in % 0.11 
Progress in % 0.11 
Progress in % 0.11 
Progress in % 0.12 
Progress in % 0.12 
Progress in % 0.13 
Progress in % 0.13 
Progress in % 0.13 
Progress in % 0.14 
Progress in % 0.14 
Progress in % 0.15 
Progress in % 0.15 
Progress in % 0.16 
Progress in % 0.16 
Progress in % 0.16 
Progress in % 0.17 
Progress in % 0.17 
Progress in % 0.18 
Progress in % 0.18 
Progress in % 0.19 
Progress in % 0.19 
Progress in % 0.19 
Progress in % 0.20 
Progress in % 0.20 
Progress in % 0.21 
Progress in % 0.21 


Progress in % 1.73 
Progress in % 1.74 
Progress in % 1.74 
Progress in % 1.75 
Progress in % 1.75 
Progress in % 1.75 
Progress in % 1.76 
Progress in % 1.76 
Progress in % 1.77 
Progress in % 1.77 
Progress in % 1.77 
Progress in % 1.78 
Progress in % 1.78 
Progress in % 1.79 
Progress in % 1.79 
Progress in % 1.80 
Progress in % 1.80 
Progress in % 1.80 
Progress in % 1.81 
Progress in % 1.81 
Progress in % 1.82 
Progress in % 1.82 
Progress in % 1.83 
Progress in % 1.83 
Progress in % 1.83 
Progress in % 1.84 
Progress in % 1.84 
Progress in % 1.85 
Progress in % 1.85 
Progress in % 1.85 
Progress in % 1.86 
Progress in % 1.86 
Progress in % 1.87 
Progress in % 1.87 
Progress in % 1.88 
Progress in % 1.88 
Progress in % 1.88 
Progress in % 1.89 
Progress in % 1.89 
Progress in % 1.90 
Progress in % 1.90 
Progress in % 1.91 
Progress in % 1.91 
Progress in % 1.91 
Progress in % 1.92 
Progress in % 1.92 
Progress in % 1.93 
Progress in % 1.93 
Progress in % 1.93 
Progress in % 1.94 


Progress in % 3.47 
Progress in % 3.48 
Progress in % 3.48 
Progress in % 3.49 
Progress in % 3.49 
Progress in % 3.49 
Progress in % 3.50 
Progress in % 3.50 
Progress in % 3.51 
Progress in % 3.51 
Progress in % 3.52 
Progress in % 3.52 
Progress in % 3.52 
Progress in % 3.53 
Progress in % 3.53 
Progress in % 3.54 
Progress in % 3.54 
Progress in % 3.55 
Progress in % 3.55 
Progress in % 3.55 
Progress in % 3.56 
Progress in % 3.56 
Progress in % 3.57 
Progress in % 3.57 
Progress in % 3.57 
Progress in % 3.58 
Progress in % 3.58 
Progress in % 3.59 
Progress in % 3.59 
Progress in % 3.60 
Progress in % 3.60 
Progress in % 3.60 
Progress in % 3.61 
Progress in % 3.61 
Progress in % 3.62 
Progress in % 3.62 
Progress in % 3.63 
Progress in % 3.63 
Progress in % 3.63 
Progress in % 3.64 
Progress in % 3.64 
Progress in % 3.65 
Progress in % 3.65 
Progress in % 3.65 
Progress in % 3.66 
Progress in % 3.66 
Progress in % 3.67 
Progress in % 3.67 
Progress in % 3.68 
Progress in % 3.68 


Progress in % 5.21 
Progress in % 5.21 
Progress in % 5.21 
Progress in % 5.22 
Progress in % 5.22 
Progress in % 5.23 
Progress in % 5.23 
Progress in % 5.24 
Progress in % 5.24 
Progress in % 5.24 
Progress in % 5.25 
Progress in % 5.25 
Progress in % 5.26 
Progress in % 5.26 
Progress in % 5.27 
Progress in % 5.27 
Progress in % 5.27 
Progress in % 5.28 
Progress in % 5.28 
Progress in % 5.29 
Progress in % 5.29 
Progress in % 5.29 
Progress in % 5.30 
Progress in % 5.30 
Progress in % 5.31 
Progress in % 5.31 
Progress in % 5.32 
Progress in % 5.32 
Progress in % 5.32 
Progress in % 5.33 
Progress in % 5.33 
Progress in % 5.34 
Progress in % 5.34 
Progress in % 5.35 
Progress in % 5.35 
Progress in % 5.35 
Progress in % 5.36 
Progress in % 5.36 
Progress in % 5.37 
Progress in % 5.37 
Progress in % 5.37 
Progress in % 5.38 
Progress in % 5.38 
Progress in % 5.39 
Progress in % 5.39 
Progress in % 5.40 
Progress in % 5.40 
Progress in % 5.40 
Progress in % 5.41 
Progress in % 5.41 


Progress in % 6.94 
Progress in % 6.94 
Progress in % 6.95 
Progress in % 6.95 
Progress in % 6.96 
Progress in % 6.96 
Progress in % 6.96 
Progress in % 6.97 
Progress in % 6.97 
Progress in % 6.98 
Progress in % 6.98 
Progress in % 6.99 
Progress in % 6.99 
Progress in % 6.99 
Progress in % 7.00 
Progress in % 7.00 
Progress in % 7.01 
Progress in % 7.01 
Progress in % 7.01 
Progress in % 7.02 
Progress in % 7.02 
Progress in % 7.03 
Progress in % 7.03 
Progress in % 7.04 
Progress in % 7.04 
Progress in % 7.04 
Progress in % 7.05 
Progress in % 7.05 
Progress in % 7.06 
Progress in % 7.06 
Progress in % 7.07 
Progress in % 7.07 
Progress in % 7.07 
Progress in % 7.08 
Progress in % 7.08 
Progress in % 7.09 
Progress in % 7.09 
Progress in % 7.09 
Progress in % 7.10 
Progress in % 7.10 
Progress in % 7.11 
Progress in % 7.11 
Progress in % 7.12 
Progress in % 7.12 
Progress in % 7.12 
Progress in % 7.13 
Progress in % 7.13 
Progress in % 7.14 
Progress in % 7.14 
Progress in % 7.15 


...

Progress in % 93.02 
Progress in % 93.03 
Progress in % 93.03 
Progress in % 93.04 
Progress in % 93.04 
Progress in % 93.04 
Progress in % 93.05 
Progress in % 93.05 
Progress in % 93.06 
Progress in % 93.06 
Progress in % 93.07 
Progress in % 93.07 
Progress in % 93.07 
Progress in % 93.08 
Progress in % 93.08 
Progress in % 93.09 
Progress in % 93.09 
Progress in % 93.09 
Progress in % 93.10 
Progress in % 93.10 
Progress in % 93.11 
Progress in % 93.11 
Progress in % 93.12 
Progress in % 93.12 
Progress in % 93.12 
Progress in % 93.13 
Progress in % 93.13 
Progress in % 93.14 
Progress in % 93.14 
Progress in % 93.15 
Progress in % 93.15 
Progress in % 93.15 
Progress in % 93.16 
Progress in % 93.16 
Progress in % 93.17 
Progress in % 93.17 
Progress in % 93.17 
Progress in % 93.18 
Progress in % 93.18 
Progress in % 93.19 
Progress in % 93.19 
Progress in % 93.20 
Progress in % 93.20 
Progress in % 93.20 
Progress in % 93.21 
Progress in % 93.21 
Progress in % 93.22 
Progress in %

Progress in % 94.72 
Progress in % 94.72 
Progress in % 94.73 
Progress in % 94.73 
Progress in % 94.73 
Progress in % 94.74 
Progress in % 94.74 
Progress in % 94.75 
Progress in % 94.75 
Progress in % 94.76 
Progress in % 94.76 
Progress in % 94.76 
Progress in % 94.77 
Progress in % 94.77 
Progress in % 94.78 
Progress in % 94.78 
Progress in % 94.79 
Progress in % 94.79 
Progress in % 94.79 
Progress in % 94.80 
Progress in % 94.80 
Progress in % 94.81 
Progress in % 94.81 
Progress in % 94.81 
Progress in % 94.82 
Progress in % 94.82 
Progress in % 94.83 
Progress in % 94.83 
Progress in % 94.84 
Progress in % 94.84 
Progress in % 94.84 
Progress in % 94.85 
Progress in % 94.85 
Progress in % 94.86 
Progress in % 94.86 
Progress in % 94.87 
Progress in % 94.87 
Progress in % 94.87 
Progress in % 94.88 
Progress in % 94.88 
Progress in % 94.89 
Progress in % 94.89 
Progress in % 94.89 
Progress in % 94.90 
Progress in % 94.90 
Progress in % 94.91 
Progress in % 94.91 
Progress in %

Progress in % 96.41 
Progress in % 96.42 
Progress in % 96.42 
Progress in % 96.43 
Progress in % 96.43 
Progress in % 96.43 
Progress in % 96.44 
Progress in % 96.44 
Progress in % 96.45 
Progress in % 96.45 
Progress in % 96.45 
Progress in % 96.46 
Progress in % 96.46 
Progress in % 96.47 
Progress in % 96.47 
Progress in % 96.48 
Progress in % 96.48 
Progress in % 96.48 
Progress in % 96.49 
Progress in % 96.49 
Progress in % 96.50 
Progress in % 96.50 
Progress in % 96.51 
Progress in % 96.51 
Progress in % 96.51 
Progress in % 96.52 
Progress in % 96.52 
Progress in % 96.53 
Progress in % 96.53 
Progress in % 96.53 
Progress in % 96.54 
Progress in % 96.54 
Progress in % 96.55 
Progress in % 96.55 
Progress in % 96.56 
Progress in % 96.56 
Progress in % 96.56 
Progress in % 96.57 
Progress in % 96.57 
Progress in % 96.58 
Progress in % 96.58 
Progress in % 96.59 
Progress in % 96.59 
Progress in % 96.59 
Progress in % 96.60 
Progress in % 96.60 
Progress in % 96.61 
Progress in %

Progress in % 98.13 
Progress in % 98.14 
Progress in % 98.14 
Progress in % 98.15 
Progress in % 98.15 
Progress in % 98.15 
Progress in % 98.16 
Progress in % 98.16 
Progress in % 98.17 
Progress in % 98.17 
Progress in % 98.17 
Progress in % 98.18 
Progress in % 98.18 
Progress in % 98.19 
Progress in % 98.19 
Progress in % 98.20 
Progress in % 98.20 
Progress in % 98.20 
Progress in % 98.21 
Progress in % 98.21 
Progress in % 98.22 
Progress in % 98.22 
Progress in % 98.23 
Progress in % 98.23 
Progress in % 98.23 
Progress in % 98.24 
Progress in % 98.24 
Progress in % 98.25 
Progress in % 98.25 
Progress in % 98.25 
Progress in % 98.26 
Progress in % 98.26 
Progress in % 98.27 
Progress in % 98.27 
Progress in % 98.28 
Progress in % 98.28 
Progress in % 98.28 
Progress in % 98.29 
Progress in % 98.29 
Progress in % 98.30 
Progress in % 98.30 
Progress in % 98.31 
Progress in % 98.31 
Progress in % 98.31 
Progress in % 98.32 
Progress in % 98.32 
Progress in % 98.33 
Progress in %

Progress in % 99.81 
Progress in % 99.82 
Progress in % 99.82 
Progress in % 99.83 
Progress in % 99.83 
Progress in % 99.84 
Progress in % 99.84 
Progress in % 99.84 
Progress in % 99.85 
Progress in % 99.85 
Progress in % 99.86 
Progress in % 99.86 
Progress in % 99.87 
Progress in % 99.87 
Progress in % 99.87 
Progress in % 99.88 
Progress in % 99.88 
Progress in % 99.89 
Progress in % 99.89 
Progress in % 99.89 
Progress in % 99.90 
Progress in % 99.90 
Progress in % 99.91 
Progress in % 99.91 
Progress in % 99.92 
Progress in % 99.92 
Progress in % 99.92 
Progress in % 99.93 
Progress in % 99.93 
Progress in % 99.94 
Progress in % 99.94 
Progress in % 99.95 
Progress in % 99.95 
Progress in % 99.95 
Progress in % 99.96 
Progress in % 99.96 
Progress in % 99.97 
Progress in % 99.97 
Progress in % 99.97 
Progress in % 99.98 
Progress in % 99.98 
Progress in % 99.99 
Progress in % 99.99 
Progress in % 100.00 
Progress in % 100.00 


In [46]:
precision_20_cos, recall_20_cos, f1_20_cos = score_metrics(n=21, reco_all_user= reco_all_user_20_cos)

In [47]:
print(precision_20_cos)
print(recall_20_cos)
print(f1_20_cos)

0.00565
0.08642
0.010606560225915064


### 50 recommendations

In [48]:
reco_all_user_50_cos = []
reco_all_user_50_cos, _ = n_hybrid_recommendation(users_test_unique, n=51, distance_matrix=item_tfidf_similarity)

Progress in % 0.00 
Progress in % 0.01 
Progress in % 0.01 
Progress in % 0.02 
Progress in % 0.02 
Progress in % 0.03 
Progress in % 0.03 
Progress in % 0.03 
Progress in % 0.04 
Progress in % 0.04 
Progress in % 0.05 
Progress in % 0.05 
Progress in % 0.05 
Progress in % 0.06 
Progress in % 0.06 
Progress in % 0.07 
Progress in % 0.07 
Progress in % 0.08 
Progress in % 0.08 
Progress in % 0.08 
Progress in % 0.09 
Progress in % 0.09 
Progress in % 0.10 
Progress in % 0.10 
Progress in % 0.11 
Progress in % 0.11 
Progress in % 0.11 
Progress in % 0.12 
Progress in % 0.12 
Progress in % 0.13 
Progress in % 0.13 
Progress in % 0.13 
Progress in % 0.14 
Progress in % 0.14 
Progress in % 0.15 
Progress in % 0.15 
Progress in % 0.16 
Progress in % 0.16 
Progress in % 0.16 
Progress in % 0.17 
Progress in % 0.17 
Progress in % 0.18 
Progress in % 0.18 
Progress in % 0.19 
Progress in % 0.19 
Progress in % 0.19 
Progress in % 0.20 
Progress in % 0.20 
Progress in % 0.21 
Progress in % 0.21 


Progress in % 1.76 
Progress in % 1.77 
Progress in % 1.77 
Progress in % 1.77 
Progress in % 1.78 
Progress in % 1.78 
Progress in % 1.79 
Progress in % 1.79 
Progress in % 1.80 
Progress in % 1.80 
Progress in % 1.80 
Progress in % 1.81 
Progress in % 1.81 
Progress in % 1.82 
Progress in % 1.82 
Progress in % 1.83 
Progress in % 1.83 
Progress in % 1.83 
Progress in % 1.84 
Progress in % 1.84 
Progress in % 1.85 
Progress in % 1.85 
Progress in % 1.85 
Progress in % 1.86 
Progress in % 1.86 
Progress in % 1.87 
Progress in % 1.87 
Progress in % 1.88 
Progress in % 1.88 
Progress in % 1.88 
Progress in % 1.89 
Progress in % 1.89 
Progress in % 1.90 
Progress in % 1.90 
Progress in % 1.91 
Progress in % 1.91 
Progress in % 1.91 
Progress in % 1.92 
Progress in % 1.92 
Progress in % 1.93 
Progress in % 1.93 
Progress in % 1.93 
Progress in % 1.94 
Progress in % 1.94 
Progress in % 1.95 
Progress in % 1.95 
Progress in % 1.96 
Progress in % 1.96 
Progress in % 1.96 
Progress in % 1.97 


Progress in % 3.50 
Progress in % 3.51 
Progress in % 3.51 
Progress in % 3.52 
Progress in % 3.52 
Progress in % 3.52 
Progress in % 3.53 
Progress in % 3.53 
Progress in % 3.54 
Progress in % 3.54 
Progress in % 3.55 
Progress in % 3.55 
Progress in % 3.55 
Progress in % 3.56 
Progress in % 3.56 
Progress in % 3.57 
Progress in % 3.57 
Progress in % 3.57 
Progress in % 3.58 
Progress in % 3.58 
Progress in % 3.59 
Progress in % 3.59 
Progress in % 3.60 
Progress in % 3.60 
Progress in % 3.60 
Progress in % 3.61 
Progress in % 3.61 
Progress in % 3.62 
Progress in % 3.62 
Progress in % 3.63 
Progress in % 3.63 
Progress in % 3.63 
Progress in % 3.64 
Progress in % 3.64 
Progress in % 3.65 
Progress in % 3.65 
Progress in % 3.65 
Progress in % 3.66 
Progress in % 3.66 
Progress in % 3.67 
Progress in % 3.67 
Progress in % 3.68 
Progress in % 3.68 
Progress in % 3.68 
Progress in % 3.69 
Progress in % 3.69 
Progress in % 3.70 
Progress in % 3.70 
Progress in % 3.71 
Progress in % 3.71 


Progress in % 5.29 
Progress in % 5.29 
Progress in % 5.29 
Progress in % 5.30 
Progress in % 5.30 
Progress in % 5.31 
Progress in % 5.31 
Progress in % 5.32 
Progress in % 5.32 
Progress in % 5.32 
Progress in % 5.33 
Progress in % 5.33 
Progress in % 5.34 
Progress in % 5.34 
Progress in % 5.35 
Progress in % 5.35 
Progress in % 5.35 
Progress in % 5.36 
Progress in % 5.36 
Progress in % 5.37 
Progress in % 5.37 
Progress in % 5.37 
Progress in % 5.38 
Progress in % 5.38 
Progress in % 5.39 
Progress in % 5.39 
Progress in % 5.40 
Progress in % 5.40 
Progress in % 5.40 
Progress in % 5.41 
Progress in % 5.41 
Progress in % 5.42 
Progress in % 5.42 
Progress in % 5.43 
Progress in % 5.43 
Progress in % 5.43 
Progress in % 5.44 
Progress in % 5.44 
Progress in % 5.45 
Progress in % 5.45 
Progress in % 5.46 
Progress in % 5.46 
Progress in % 5.46 
Progress in % 5.47 
Progress in % 5.47 
Progress in % 5.48 
Progress in % 5.48 
Progress in % 5.48 
Progress in % 5.49 
Progress in % 5.49 


Progress in % 7.01 
Progress in % 7.02 
Progress in % 7.02 
Progress in % 7.03 
Progress in % 7.03 
Progress in % 7.04 
Progress in % 7.04 
Progress in % 7.04 
Progress in % 7.05 
Progress in % 7.05 
Progress in % 7.06 
Progress in % 7.06 
Progress in % 7.07 
Progress in % 7.07 
Progress in % 7.07 
Progress in % 7.08 
Progress in % 7.08 
Progress in % 7.09 
Progress in % 7.09 
Progress in % 7.09 
Progress in % 7.10 
Progress in % 7.10 
Progress in % 7.11 
Progress in % 7.11 
Progress in % 7.12 
Progress in % 7.12 
Progress in % 7.12 
Progress in % 7.13 
Progress in % 7.13 
Progress in % 7.14 
Progress in % 7.14 
Progress in % 7.15 
Progress in % 7.15 
Progress in % 7.15 
Progress in % 7.16 
Progress in % 7.16 
Progress in % 7.17 
Progress in % 7.17 
Progress in % 7.18 
Progress in % 7.18 
Progress in % 7.18 
Progress in % 7.19 
Progress in % 7.19 
Progress in % 7.20 
Progress in % 7.20 
Progress in % 7.20 
Progress in % 7.21 
Progress in % 7.21 
Progress in % 7.22 
Progress in % 7.22 


...

Progress in % 92.88 
Progress in % 92.89 
Progress in % 92.89 
Progress in % 92.90 
Progress in % 92.90 
Progress in % 92.91 
Progress in % 92.91 
Progress in % 92.91 
Progress in % 92.92 
Progress in % 92.92 
Progress in % 92.93 
Progress in % 92.93 
Progress in % 92.93 
Progress in % 92.94 
Progress in % 92.94 
Progress in % 92.95 
Progress in % 92.95 
Progress in % 92.96 
Progress in % 92.96 
Progress in % 92.96 
Progress in % 92.97 
Progress in % 92.97 
Progress in % 92.98 
Progress in % 92.98 
Progress in % 92.99 
Progress in % 92.99 
Progress in % 92.99 
Progress in % 93.00 
Progress in % 93.00 
Progress in % 93.01 
Progress in % 93.01 
Progress in % 93.01 
Progress in % 93.02 
Progress in % 93.02 
Progress in % 93.03 
Progress in % 93.03 
Progress in % 93.04 
Progress in % 93.04 
Progress in % 93.04 
Progress in % 93.05 
Progress in % 93.05 
Progress in % 93.06 
Progress in % 93.06 
Progress in % 93.07 
Progress in % 93.07 
Progress in % 93.07 
Progress in % 93.08 
Progress in %

Progress in % 94.57 
Progress in % 94.58 
Progress in % 94.58 
Progress in % 94.59 
Progress in % 94.59 
Progress in % 94.60 
Progress in % 94.60 
Progress in % 94.60 
Progress in % 94.61 
Progress in % 94.61 
Progress in % 94.62 
Progress in % 94.62 
Progress in % 94.63 
Progress in % 94.63 
Progress in % 94.63 
Progress in % 94.64 
Progress in % 94.64 
Progress in % 94.65 
Progress in % 94.65 
Progress in % 94.65 
Progress in % 94.66 
Progress in % 94.66 
Progress in % 94.67 
Progress in % 94.67 
Progress in % 94.68 
Progress in % 94.68 
Progress in % 94.68 
Progress in % 94.69 
Progress in % 94.69 
Progress in % 94.70 
Progress in % 94.70 
Progress in % 94.71 
Progress in % 94.71 
Progress in % 94.71 
Progress in % 94.72 
Progress in % 94.72 
Progress in % 94.73 
Progress in % 94.73 
Progress in % 94.73 
Progress in % 94.74 
Progress in % 94.74 
Progress in % 94.75 
Progress in % 94.75 
Progress in % 94.76 
Progress in % 94.76 
Progress in % 94.76 
Progress in % 94.77 
Progress in %

Progress in % 96.24 
Progress in % 96.25 
Progress in % 96.25 
Progress in % 96.26 
Progress in % 96.26 
Progress in % 96.26 
Progress in % 96.27 
Progress in % 96.27 
Progress in % 96.28 
Progress in % 96.28 
Progress in % 96.29 
Progress in % 96.29 
Progress in % 96.29 
Progress in % 96.30 
Progress in % 96.30 
Progress in % 96.31 
Progress in % 96.31 
Progress in % 96.32 
Progress in % 96.32 
Progress in % 96.32 
Progress in % 96.33 
Progress in % 96.33 
Progress in % 96.34 
Progress in % 96.34 
Progress in % 96.35 
Progress in % 96.35 
Progress in % 96.35 
Progress in % 96.36 
Progress in % 96.36 
Progress in % 96.37 
Progress in % 96.37 
Progress in % 96.37 
Progress in % 96.38 
Progress in % 96.38 
Progress in % 96.39 
Progress in % 96.39 
Progress in % 96.40 
Progress in % 96.40 
Progress in % 96.40 
Progress in % 96.41 
Progress in % 96.41 
Progress in % 96.42 
Progress in % 96.42 
Progress in % 96.43 
Progress in % 96.43 
Progress in % 96.43 
Progress in % 96.44 
Progress in %

Progress in % 97.93 
Progress in % 97.93 
Progress in % 97.93 
Progress in % 97.94 
Progress in % 97.94 
Progress in % 97.95 
Progress in % 97.95 
Progress in % 97.96 
Progress in % 97.96 
Progress in % 97.96 
Progress in % 97.97 
Progress in % 97.97 
Progress in % 97.98 
Progress in % 97.98 
Progress in % 97.98 
Progress in % 97.99 
Progress in % 97.99 
Progress in % 98.00 
Progress in % 98.00 
Progress in % 98.01 
Progress in % 98.01 
Progress in % 98.01 
Progress in % 98.02 
Progress in % 98.02 
Progress in % 98.03 
Progress in % 98.03 
Progress in % 98.04 
Progress in % 98.04 
Progress in % 98.04 
Progress in % 98.05 
Progress in % 98.05 
Progress in % 98.06 
Progress in % 98.06 
Progress in % 98.07 
Progress in % 98.07 
Progress in % 98.07 
Progress in % 98.08 
Progress in % 98.08 
Progress in % 98.09 
Progress in % 98.09 
Progress in % 98.09 
Progress in % 98.10 
Progress in % 98.10 
Progress in % 98.11 
Progress in % 98.11 
Progress in % 98.12 
Progress in % 98.12 
Progress in %

Progress in % 99.60 
Progress in % 99.60 
Progress in % 99.61 
Progress in % 99.61 
Progress in % 99.62 
Progress in % 99.62 
Progress in % 99.62 
Progress in % 99.63 
Progress in % 99.63 
Progress in % 99.64 
Progress in % 99.64 
Progress in % 99.65 
Progress in % 99.65 
Progress in % 99.65 
Progress in % 99.66 
Progress in % 99.66 
Progress in % 99.67 
Progress in % 99.67 
Progress in % 99.68 
Progress in % 99.68 
Progress in % 99.68 
Progress in % 99.69 
Progress in % 99.69 
Progress in % 99.70 
Progress in % 99.70 
Progress in % 99.70 
Progress in % 99.71 
Progress in % 99.71 
Progress in % 99.72 
Progress in % 99.72 
Progress in % 99.73 
Progress in % 99.73 
Progress in % 99.73 
Progress in % 99.74 
Progress in % 99.74 
Progress in % 99.75 
Progress in % 99.75 
Progress in % 99.76 
Progress in % 99.76 
Progress in % 99.76 
Progress in % 99.77 
Progress in % 99.77 
Progress in % 99.78 
Progress in % 99.78 
Progress in % 99.79 
Progress in % 99.79 
Progress in % 99.79 
Progress in %

In [49]:
precision_50_cos, recall_50_cos, f1_50_cos = score_metrics(n=51, reco_all_user= reco_all_user_50_cos)

In [50]:
print(precision_50_cos)
print(recall_50_cos)
print(f1_50_cos)

0.00373
0.14258
0.00726981614380425


# MARK

Busca medir el recall ponderado por la posición en la que está el ítem en nuestra predicción. Es decir, si el item del usuario en test yo lo recomiendo en la posición 3 el TP no vale 1, vale 1/3.

In [51]:
#Si aparece error de recmetrics de signature
#Instalar esto:
#!pip install funcsigs
#Entrar al archivo \anaconda3\lib\site-packages\recmetrics\plots.py
#Cambiar donde dice  from sklearn.utils.fixes import signature por from funcsigs import signature

In [52]:
import recmetrics

Calculamos el mark sobre 5 recomendaciones que es el mejor F1.

Lo que buscamos es que el mark del hibrido > a mark del content. Si esto ocurre podemos recomendar el hibrido.

In [53]:
#Estos son los item en test para cada usuario
list_test_item_per_user = []
for user in users_test_unique:
    list_test_item_per_user.append(X_test[X_test["user_id"]==user]["item_id"].values.tolist())
list_test_item_per_user[1:10]

[[154002],
 [1111981],
 [734229],
 [512791],
 [2958376],
 [124204],
 [1519172],
 [2098200, 766885, 2261828],
 [1780063]]

In [54]:
mark_5 = recmetrics.mark(list_test_item_per_user, reco_all_user_5_cos, k=5)
print(mark_5)
print(recall_5_cos)

0.017874457231988532
0.03503


In [55]:
mark_10 = recmetrics.mark(list_test_item_per_user, reco_all_user_10_cos, k=10)
print(mark_10)
print(recall_10_cos)

0.01907727793045655
0.05506


In [56]:
mark_20 = recmetrics.mark(list_test_item_per_user, reco_all_user_20_cos, k=20)
print(mark_20)
print(recall_20_cos)

0.01923883727345896
0.08642


In [57]:
mark_50 = recmetrics.mark(list_test_item_per_user, reco_all_user_50_cos, k=50)
print(mark_50)
print(recall_50_cos)

0.014002366375996646
0.14258


# Resultados

In [58]:
resultados = {'n_recomendaciones': [5, 10, 20, 50],
              'hybrid_recall': [recall_5_cos,
                            recall_10_cos,
                            recall_20_cos,
                            recall_50_cos],
              'hybrid_precision': [precision_5_cos,
                               precision_10_cos,
                               precision_20_cos,
                               precision_50_cos],
              'hybrid_F1': [f1_5_cos,
                            f1_10_cos,
                            f1_20_cos,
                            f1_50_cos],                          
              'MAR@K_hybrid': [mark_5,
                            mark_10,
                            mark_20,
                            mark_50]
                    }

In [59]:
resultados_df = pd.DataFrame.from_dict(resultados)
resultados_df

,n_recomendaciones,hybrid_recall,hybrid_precision,hybrid_F1,MAR@K_hybrid
0,5,0.03503,0.00916,0.014523,0.017874
1,10,0.05506,0.00720,0.012735,0.019077
2,20,0.08642,0.00565,0.010607,0.019239
3,50,0.14258,0.00373,0.007270,0.014002


In [60]:
resultados_df.to_csv("resultados_hybrid_fine_tunning.csv",index=False)